In [1]:
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cmaps
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np
import metpy.calc as mycalc

In [2]:
RV=xr.open_dataset('./RV850_temp.nc')['vo']
RV

<xarray.DataArray 'vo' (time: 120, latitude: 37, longitude: 37)>
[164280 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 109.0 109.2 109.5 109.8 ... 117.5 117.8 118.0
  * latitude   (latitude) float32 26.0 25.75 25.5 25.25 ... 17.5 17.25 17.0
    level      int32 ...
  * time       (time) datetime64[ns] 2018-09-01 ... 2018-09-30T18:00:00
Attributes:
    units:          s**-1
    long_name:      Vorticity (relative)
    standard_name:  atmosphere_relative_vorticity

In [3]:
SP=xr.open_dataset('./SP_temp.nc')['sp']
SP

<xarray.DataArray 'sp' (time: 120, expver: 2, latitude: 37, longitude: 37)>
[328560 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 109.0 109.2 109.5 109.8 ... 117.5 117.8 118.0
  * latitude   (latitude) float32 26.0 25.75 25.5 25.25 ... 17.5 17.25 17.0
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 2018-09-01 ... 2018-09-30T18:00:00
Attributes:
    units:          Pa
    long_name:      Surface pressure
    standard_name:  surface_air_pressure

In [4]:
timelist=SP.time.data
num=0
for i in timelist:
    if '2018-09-16T06' in i.__str__() :
        print(num)
    num+=1

61


In [5]:
timelist[61].__str__()

'2018-09-16T06:00:00.000000000'

In [6]:
timeselect=np.arange(61-3,61+4,1)
timeselect

array([58, 59, 60, 61, 62, 63, 64])

In [7]:
# levels=np.arange(6,61,2).tolist()
# labels =[]
# for i in levels:
#     labels.append(str(i))
# labels=tuple(labels)
# levels1=[0]+np.arange(6,61,2).tolist()

In [8]:
lat=SP.latitude
lon=SP.longitude
levels=[0,7,10,13,16,19,22,25,28,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,69.4,74.7,80,85.4,90.7,96,100,105,110,115,120,125,130,135,140,145,150,155,160]
colors=['#ffffff','#7ffdfc','#6feff8','#5be1de','#54cbd1','#38c0c3','#38a4c1','#219ea6','#108b95','#097595','#00b62a','#3dbc3d','#6dcc5b','#97e359','#d4ed70','#fffc89','#fdd952','#fea549','#ff6728','#fb370d','#f30b01','#d3010c','#a60402','#850201','#5e060b','#ae01fa','#b223f7','#c247fe','#d66af8','#de92fa','#f4b4fa','#fdd5fd','#fbcaea','#f8b7d8','#ffa2be','#ee9db3','#f88a96','#f6757a','#ff6172','#fb5d5b','#e2494e','#c63d51','#cc3a51','#b72347','#981b45']
box1=[109,118,17,26]
for t_i in timeselect:
    fig = plt.figure(figsize=(10, 10))
    ax = fig.subplots(1, 1, subplot_kw={'projection': ccrs.PlateCarree(central_longitude=113.5)})
    ax.coastlines('50m')
    ax.set_extent(box1,crs=ccrs.PlateCarree())

    # 标注坐标轴
    ax.set_xticks(np.arange(box1[0], box1[1] + 1, 1), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(box1[2], box1[3] + 1, 1), crs=ccrs.PlateCarree())

    # 经纬度格式，把0经度设置不加E和W
    lon_formatter = LongitudeFormatter(zero_direction_label=False)
    lat_formatter = LatitudeFormatter()
    ax.gridlines(xlocs=np.arange(box1[0], box1[1] + 1, 1), ylocs=np.arange(box1[2], box1[3] + 1, 1), linestyle=':', linewidth=0.3, x_inline=False, y_inline=False, color='k')
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)

    # 绘制
    smoothSP=mycalc.smooth_gaussian(SP.loc[timelist[t_i].__str__(),1,:,:]/100,9)
    line=ax.contour(lon,lat,smoothSP,levels=np.arange(892,1006,4),colors='#5d122b',transform=ccrs.PlateCarree(),alpha=0.5,)
    ax.clabel(line, inline=True, fontsize=10, fmt='%.0f')
    colorbar=ax.contourf(lon,lat,RV.loc[timelist[t_i].__str__(),:,:]*10**5,levels=levels,colors=colors,transform=ccrs.PlateCarree())
    cb=plt.colorbar(colorbar,extendrect='True',pad=0.005,fraction=0.15, shrink=0.83,aspect=30,location='right')
    cb.set_ticks([7,16,25,34,40,46,52,58,64,80,96,110,125,140,155])
    cb.set_ticklabels(('7','16','25','34','40','46','52','58','64','80','96','110','125','140','155'))
    
    plt.savefig('./tygif/'+str(t_i)+'.png',dpi=300)
    plt.close()


In [9]:
# 动图制作
from PIL import Image
import os
imgFiles = [fn for fn in os.listdir('./tygif/') if fn.endswith('.png')]
imgFiles.sort()
print(imgFiles)

['58.png', '59.png', '60.png', '61.png', '62.png', '63.png', '64.png']


In [10]:
images = [Image.open('./tygif/'+fn) for fn in imgFiles]
im = images[0]
filename = './tygif/test.gif'
im.save(fp=filename, format='gif', save_all=True, append_images=images[1:], duration=100)

In [2]:
# 动图制作
from PIL import Image
import os
imgFiles = [fn for fn in os.listdir('D:\picture\亚欧\\500hPa\\2022\\11\\') if fn.endswith('.jpg')]
imgFiles.sort()
print(imgFiles)

['2022_11_19_0800.jpg', '2022_11_19_2000.jpg', '2022_11_20_0800.jpg', '2022_11_20_2000.jpg', '2022_11_21_0800.jpg', '2022_11_21_2000.jpg', '2022_11_22_0800.jpg', '2022_11_22_2000.jpg', '2022_11_23_0800.jpg', '2022_11_23_2000.jpg', '2022_11_24_0800.jpg', '2022_11_24_2000.jpg', '2022_11_25_0800.jpg', '2022_11_25_2000.jpg', '2022_11_26_0800.jpg']


In [6]:
images = [Image.open('D:\picture\亚欧\\500hPa\\2022\\11\\'+fn) for fn in imgFiles]
im = images[8]
filename = 'D:\picture\亚欧\\500hPa\\2022\\11\\test.gif'
im.save(fp=filename, format='gif', save_all=True, append_images=images[9:], duration=500)

In [4]:
help(im.save)

Help on method save in module PIL.Image:

save(fp, format=None, **params) method of PIL.JpegImagePlugin.JpegImageFile instance
    Saves this image under the given filename.  If no format is
    specified, the format to use is determined from the filename
    extension, if possible.
    
    Keyword options can be used to provide additional instructions
    to the writer. If a writer doesn't recognise an option, it is
    silently ignored. The available options are described in the
    :doc:`image format documentation
    <../handbook/image-file-formats>` for each writer.
    
    You can use a file object instead of a filename. In this case,
    you must always specify the format. The file object must
    implement the ``seek``, ``tell``, and ``write``
    methods, and be opened in binary mode.
    
    :param fp: A filename (string), pathlib.Path object or file object.
    :param format: Optional format override.  If omitted, the
       format to use is determined from the filename e